<a href="https://colab.research.google.com/github/yumnaehab-tech/data-science-projects/blob/main/text_data%20%3D%20pd.read_csv('/content/IMDB-Dataset.csv').ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Text + Image Classification


In [2]:
import pandas as pd
import zipfile
from google.colab import files

In [3]:

uploaded = files.upload()

Saving archive (5).zip to archive (5).zip


In [4]:
for fn in uploaded.keys():
    if fn.endswith(".zip"):
        with zipfile.ZipFile(fn, 'r') as zip_ref:
            zip_ref.extractall("data")

        csv_path = "data/IMDB Dataset.csv"
    else:

        csv_path = fn

In [5]:
# Preprocessing function
def preprocess_text(text):
    import re
    import string
    from nltk.corpus import stopwords
    stop_words = set(stopwords.words('english'))

    text = text.lower()  # lowercase
    text = re.sub(f"[{string.punctuation}]", "", text)  # remove punctuation
    text = " ".join([word for word in text.split() if word not in stop_words])  # remove stopwords
    return text

In [6]:


# IMPORTS إضافية للـ Text Preprocessing

import re
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords


# Preprocessing function

stop_words = set(stopwords.words('english'))  # تعريف stopwords مرة واحدة

def preprocess_text(text):
    try:
        text = str(text).lower()  # تحويل النص لصغير
        text = re.sub(f"[{string.punctuation}]", "", text)  # إزالة الرموز
        text = " ".join([word for word in text.split() if word not in stop_words])  # إزالة stopwords
        return text
    except:
        return ""


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [7]:


import zipfile
import pandas as pd
import os
zip_path = "/content/archive (5).zip"


with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall("/content")

print(" الملفات بعد فك الضغط:")
print(os.listdir("/content"))


csv_path = "/content/IMDB Dataset.csv"

# قراءة الداتا باستخدام pandas
text_data = pd.read_csv(csv_path)

#  عرض أول 5 صفوف + أسماء الأعمدة للتأكد إن كل حاجة تمام
print("\n أول 5 صفوف من البيانات:")
print(text_data.head())

print("\n أسماء الأعمدة:")
print(text_data.columns)


 الملفات بعد فك الضغط:
['.config', 'IMDB Dataset.csv', 'archive (5).zip', 'data', 'sample_data']

 أول 5 صفوف من البيانات:
                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive

 أسماء الأعمدة:
Index(['review', 'sentiment'], dtype='object')


In [8]:
text_data.columns


Index(['review', 'sentiment'], dtype='object')

In [9]:

# معالجة النصوص وتدريب نموذج التصنيف


# (Preprocessing)
import re
import string
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # تحويل كل الكلام لحروف صغيرة
    text = text.lower()
    # إزالة علامات الترقيم
    text = re.sub(f"[{string.punctuation}]", "", text)
    # حذف الـ stopwords (الكلمات الشائعة اللي مالهاش معنى تحليلي)
    text = " ".join([word for word in text.split() if word not in stop_words])
    return text

#  تطبيق التنظيف على عمود المراجعات
text_data['cleaned_review'] = text_data['review'].apply(preprocess_text)

#  فصل الخصائص (X) عن التصنيفات (y)
X_text = text_data['cleaned_review']
y_text = text_data['sentiment']

# تحويل النصوص لأرقام باستخدام TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(max_features=5000)
X_text_features = tfidf.fit_transform(X_text)

#  تقسيم البيانات لتدريب واختبار
from sklearn.model_selection import train_test_split
X_train_text, X_test_text, y_train_text, y_test_text = train_test_split(
    X_text_features, y_text, test_size=0.2, random_state=42
)

#  بناء وتدريب الموديل
from sklearn.linear_model import LogisticRegression
text_model = LogisticRegression(max_iter=1000)
text_model.fit(X_train_text, y_train_text)

#  التنبؤ والتقييم
from sklearn.metrics import accuracy_score, classification_report
y_pred_text = text_model.predict(X_test_text)

print(" Text Classification Accuracy:", accuracy_score(y_test_text, y_pred_text))
print("\n Classification Report:\n", classification_report(y_test_text, y_pred_text))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


 Text Classification Accuracy: 0.8881

 Classification Report:
               precision    recall  f1-score   support

    negative       0.90      0.87      0.89      4961
    positive       0.88      0.90      0.89      5039

    accuracy                           0.89     10000
   macro avg       0.89      0.89      0.89     10000
weighted avg       0.89      0.89      0.89     10000



In [10]:
# Step 3 - Split data into training and testing sets
from sklearn.model_selection import train_test_split

X = text_data['cleaned_review']
y = text_data['sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(" Data successfully split!")
print("Training samples:", len(X_train))
print("Testing samples:", len(X_test))


 Data successfully split!
Training samples: 40000
Testing samples: 10000


In [11]:
# Step 4 - تحويل النصوص لأرقام باستخدام TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

# نحدد أقصى عدد من الكلمات المميزة اللي الموديل هيستخدمها
tfidf = TfidfVectorizer(max_features=5000)

# نحول النصوص (بعد التنظيف) إلى مميزات رقمية
X_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

print(" Text successfully vectorized using TF-IDF!")
print("Shape of training data:", X_tfidf.shape)
print("Shape of test data:", X_test_tfidf.shape)


 Text successfully vectorized using TF-IDF!
Shape of training data: (40000, 5000)
Shape of test data: (10000, 5000)


In [12]:
#  تدريب نموذج التصنيف (Logistic Regression)
from sklearn.linear_model import LogisticRegression

# نختار نوع الموديل
model = LogisticRegression(max_iter=1000)  # الماكس إتريشن عشان الموديل يتدرب كويس

# ندرّب الموديل على بيانات التدريب
model.fit(X_tfidf, y_train)

print(" Model training completed successfully!")


 Model training completed successfully!


In [13]:
# تقييم أداء الموديل
from sklearn.metrics import accuracy_score, classification_report

# نستخدم الموديل في التوقع على بيانات الاختبار
y_pred = model.predict(X_test_tfidf)

# نحسب الدقة
accuracy = accuracy_score(y_test, y_pred)
print(" Model Evaluation Completed!")
print(" Accuracy:", accuracy)

# نطبع تقرير تفصيلي عن الأداء
print("\n📋 Classification Report:")
print(classification_report(y_test, y_pred))


 Model Evaluation Completed!
 Accuracy: 0.8887

📋 Classification Report:
              precision    recall  f1-score   support

    negative       0.90      0.87      0.89      4961
    positive       0.88      0.90      0.89      5039

    accuracy                           0.89     10000
   macro avg       0.89      0.89      0.89     10000
weighted avg       0.89      0.89      0.89     10000



In [14]:
#  تجربة الموديل على جمل جديدة
def predict_sentiment(text):
    # نطبق نفس خطوات التنظيف اللي عملناها قبل كده
    cleaned = preprocess_text(text)
    # نحول النص لـ TF-IDF زي اللي اتدرب عليه الموديل
    text_tfidf = tfidf.transform([cleaned])
    # نخلي الموديل يتنبأ
    prediction = model.predict(text_tfidf)[0]
    return prediction

# نجرب شوية أمثلة
sample_reviews = [
    "I loved this movie, it was amazing and emotional!",
    "The film was boring and too long.",
    "The acting was great but the story was weak."
]

for review in sample_reviews:
    print(f" Review: {review}")
    print(f"Predicted Sentiment: {predict_sentiment(review)}\n")


 Review: I loved this movie, it was amazing and emotional!
Predicted Sentiment: positive

 Review: The film was boring and too long.
Predicted Sentiment: negative

 Review: The acting was great but the story was weak.
Predicted Sentiment: negative



In [15]:
#  Test the model with a custom review
def predict_sentiment(review):
    # Apply the same preprocessing used during training
    cleaned = preprocess_text(review)

    # Convert text into TF-IDF vector
    review_vector = tfidf.transform([cleaned])

    # Make prediction
    prediction = text_model.predict(review_vector)[0]

    print(" Review:", review)
    print(" Predicted Sentiment:", prediction)

#  Example:
predict_sentiment("The movie was amazing and I loved every moment of it!")

#  Or write your own review here:
your_sentence = input("Write your own review to test: ")
predict_sentiment(your_sentence)


 Review: The movie was amazing and I loved every moment of it!
 Predicted Sentiment: positive
Write your own review to test: I didn't like the characters or the ending
 Review: I didn't like the characters or the ending
 Predicted Sentiment: positive
